In [ ]:
import pandas as pd
import hvplot.pandas
import panel as pn
from datetime import date, datetime, timedelta
from infra_copel import MongoHistoricoOficial
from bokeh.models.formatters import DatetimeTickFormatter
from exercicio_holoviz import lista_usinas

mongo = MongoHistoricoOficial()
data_pesquisa_inicial = date.today() - timedelta(days=7)


def filtrar_data(data: date, usina: str):
    resultado = mongo["ons_geracao_usinas"].find(
        {
            "usina": usina,
            "periodo": {"$gte": datetime(data.year, data.month, data.day)},
        }
    )
    lista = list(resultado)
    df = pd.DataFrame(lista)

    if not df.empty:
        return df.hvplot(
            x="periodo", y="geracao_mwm", xformatter=DatetimeTickFormatter(days="%d/%m/%Y")
        )
    else:
        return df.hvplot()


filtro_data = pn.widgets.DatePicker(name="Data", value=data_pesquisa_inicial)
filtro_usina = pn.widgets.Select(name="Usina", options=lista_usinas(), value="Itaipu")

filtered_view = pn.Column(
    pn.Row(filtro_usina, filtro_data),
    pn.panel(
        pn.bind(filtrar_data, data=filtro_data, usina=filtro_usina),
        width=800,
        height=600,
    ),
)

filtered_view